In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image

In [2]:
train_dir = 'autoplius/train'
validation_dir = 'autoplius/test'

train_datagen = ImageDataGenerator(
    rescale = 1./255,      # 255 maximali reiksme RGB spalvu skaleja
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range =0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'categorical'
)

Found 6245 images belonging to 3 classes.
Found 2678 images belonging to 3 classes.


In [4]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation = 'relu' ),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

In [5]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer = Adam(learning_rate=0.0001), # mokimosi greitis
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [17]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

# model.save('vegetables.h5')

Epoch 1/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 174s 556ms/step - accuracy: 0.8751 - loss: 0.3316 - val_accuracy: 0.9079 - val_loss: 0.2323
Epoch 2/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 320us/step - accuracy: 0.9500 - loss: 0.1919 - val_accuracy: 0.9444 - val_loss: 0.2899
Epoch 3/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 250s 787ms/step - accuracy: 0.9030 - loss: 0.2730 - val_accuracy: 0.9267 - val_loss: 0.2007
Epoch 4/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 310us/step - accuracy: 1.0000 - loss: 0.0797 - val_accuracy: 0.9444 - val_loss: 0.2124
Epoch 5/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 245s 771ms/step - accuracy: 0.9023 - loss: 0.2657 - val_accuracy: 0.9271 - val_loss: 0.2287
Epoch 6/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 508us/step - accuracy: 0.8000 - loss: 0.5463 - val_accuracy: 1.0000 - val_loss: 0.0357
Epoch 7/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 235s 737ms/step - accuracy: 0.9110 - loss: 0.2481 - val_accuracy: 0.9271 - val_loss: 0.2022
Epoch 8/20
312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 120us/step - accuracy: 0.9500 - loss: 0

In [18]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    img = Image.open(image_path)
    if image_path.endswith(".png"):
        im = Image.open(image_path)
        img = im.convert('RGB')
    img = img.resize((150,150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

class_indices = train_generator.class_indices
class_labels = {value: key for key, value in class_indices.items()}


def predict_vegetable(path_to_img):
    img_array = preprocess_image(path_to_img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return class_labels[predicted_class[0]]

print(predict_vegetable('automobilis.jpg'))
print(predict_vegetable('automobilis2.jpg'))
print(predict_vegetable('motociklas.jpg'))
print(predict_vegetable('motociklas2.jpg'))
print(predict_vegetable('traktorius.jpg'))
print(predict_vegetable('traktorius2.jpg'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
automobiliai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
automobiliai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
motociklai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
motociklai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
motociklai
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
automobiliai


In [19]:
test_loss, test_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(test_loss)
print(test_accuracy)

133/133 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.9373 - loss: 0.2227
0.23016531765460968
0.9330826997756958
